In [1]:
import pandas as pd
from parse import parse_htmls
from util import read_place_desc, read_htmls_in, write_places_to_tsv, read_places
from service import PlaceService, SearchEngine
from index import preprocess, Index, TfIdfIndex

import requests
from tqdm import tqdm
import os
import time
import util

# Data collection
##  1.1. Get the list of places





In [ ]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

## 1.2. Crawl places

In [ ]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# 1.3 Parse Pages

In [3]:
all_htmls = read_htmls_in('./pages')

In [4]:
all_places = parse_htmls(all_htmls)

In [263]:
%store htmls

Stored 'htmls' (list)


In [4]:
%store -r htmls

In [2]:
#tsv_path = write_places_to_tsv('./', all_places)
names, descriptions = read_place_desc('./places.tsv')
place_service = PlaceService()
place_service.load('./places.tsv')

# Create Conjunctive Index

In [3]:
index = Index.create_from(names, descriptions)

# Create Search Engine

In [4]:
search_engine = SearchEngine(index, place_service)

# Run Search

In [5]:
search_engine.query('american museum')

,name,desc,url
1072,Early Television Museum,Steve McVoy was always fascinated by TV. In mi...,https://www.atlasobscura.com/places/early-tele...
4954,Madam C.J. Walker Museum &amp; WERD Radio,"This small brick building, located just a few ...",https://www.atlasobscura.com/places/madam-cj-w...
569,American Museum of Magic,"Presto Change-o! Founded in 1978, the American...",https://www.atlasobscura.com/places/american-m...
4571,Deutsches Jagd und Fischereimuseum (German Mus...,With its rows of taxidermy animal heads and di...,https://www.atlasobscura.com/places/deutsches-...
904,Lunch Box Museum,Lunch boxes bring back a certain sense of nost...,https://www.atlasobscura.com/places/lunch-box-...
...,...,...,...
3250,Bare-Chested George Washington,"At first glance, the Smithsonian’s bare-cheste...",https://www.atlasobscura.com/places/bare-chest...
7118,Castle Post,Looking more like an abandoned Medieval Times ...,https://www.atlasobscura.com/places/castle-post
5429,Old Time Wooden Nickel Company,"The adage goes, “don’t take any wooden nickels...",https://www.atlasobscura.com/places/old-time-w...
456,Mildred E. Mathias Botanical Garden,"Founded in 1929, the botanical garden at Unive...",https://www.atlasobscura.com/places/mildred-e-...


# Create TF-IDF Index

In [6]:
tf_idf_index = TfIdfIndex.create_from(names, descriptions)

# Create TF-IDF Search Engine

In [ ]:
tf_idf_search_engine = SearchEngine(tf_idf_index, place_service)

In [8]:
tf_idf_search_engine.query_top_k("american museum", 10)

,name,desc,url,similarity
4953,Siriraj Medical Museum,The Siriraj Medical Museum abounds with medica...,https://www.atlasobscura.com/places/siriraj-me...,0.327173
6562,Harvard Museum of Natural History,Collecting three different institutions into o...,https://www.atlasobscura.com/places/harvard-mu...,0.304600
991,Milwaukee Art Museum,Like the Guggenheim in New York and the Oaklan...,https://www.atlasobscura.com/places/milwaukee-...,0.294030
155,National World War II Museum,"Perhaps once thought too narrowly focused, thi...",https://www.atlasobscura.com/places/national-w...,0.294030
7194,National Cryptologic Museum,Update as of October 2021: Currently closed fo...,https://www.atlasobscura.com/places/national-c...,0.294030
6638,Thorne Miniature Rooms,In the depths of the Museum of the Art Institu...,https://www.atlasobscura.com/places/thorne-min...,0.281102
5517,Niles Essanay Silent Film Museum,It was Spring in San Francisco. One quiet Apri...,https://www.atlasobscura.com/places/niles-essa...,0.281102
3530,Oscar Getz Museum of Whiskey History,If there is a place on earth that knows its wh...,https://www.atlasobscura.com/places/oscar-getz...,0.281102
5764,National Museum of the Pacific War,Dedicated specifically to remembering the stor...,https://www.atlasobscura.com/places/national-m...,0.281102
5778,Zwack Unicum Museum,Sometimes referred to as the Hungarian nationa...,https://www.atlasobscura.com/places/zwack-unic...,0.000000
